# Метод опорных векторов

In [ ]:
import numpy
import pylab
import numpy.random
import sklearn.datasets
import sklearn.decomposition
import sklearn.cross_validation as cv
import sklearn.svm
import pandas as pd
import matplotlib as plt
from sklearn import metrics, datasets
from sklearn.cross_validation import train_test_split

%pylab inline

Функция `select_model` принимает на вход обучающую выборку и возвращает модель (метод опорных векторов) с наилучшими параметрами (accuracy) для данной выборки. Эту функцию нужно реализовать.

Подбираемые параметры подели включают в себя:
- различные функции ядра (линейное, RBF, полиномиальные разных степеней)
- различные значения константы $C$ ($ 0.1, 1, 10$)

Подбирать параметры необходимо с помощью 10-fold кросс-валидации. Сейчас в качестве заглушки функция всегда возвращает линейный SVC.

In [ ]:
def select_model(x, y):
    """
    Implement some model selection strategy here:
    seek through different kernels and parameters.

    Use a validation scheme to select the best model
    
    Quality metric: accuracy

    Returns:
        SVM classifier implemented by sklearn SVC class.
    """
    model = sklearn.svm.SVC(kernel='linear' , C=1)
    model.fit(x, y)
    return model

Далее две вспомогательные функции, которые отображают данные и разделяющую поверхность

In [ ]:
def plot_data_set(x, y, description=''):
    print "Plotting data set points"
    pylab.figure(figsize=(8, 8))

    colors = numpy.array(['r', 'b'])[y]
    pylab.title(description, fontsize='small')
    pylab.scatter(x[:, 0], x[:, 1], marker='o', c=colors, s=50)
    
def plot_decision_region(x1_min, x2_min, x1_max, x2_max, clf, n_points=1000):
    print "Plotting decision region"
    x1, x2 = numpy.meshgrid(numpy.linspace(x1_min, x1_max, n_points), numpy.linspace(x2_min, x2_max, n_points))
    z = clf.decision_function(numpy.c_[x1.ravel(), x2.ravel()]).reshape(x1.shape)

    pylab.contour(x1, x2, z, levels=[0.0], linestyles='solid', linewidths=2.0)
    pylab.contour(x1, x2, z, levels=[-1.0, 1.0], linestyles='dashed', linewidths=1.0)

Применим реализованную функцию `select_model` для нескольких синтетических и одного реального набора данных.

## Данные с линейной границей

In [ ]:
def generate_linear(size=100, k=1.1, b=0.0, nl=0.1):
    print "Generating 'Linearly-separated' data set"

    x = numpy.random.random((size, 2))
    y = numpy.zeros(size, dtype=int)
    noise = numpy.random.randn(size) * nl
    y[x[:, 1] - (k * x[:, 0] + b) > noise] = 1

    return x, y

x, y = generate_linear()
clf = select_model(x, y)
plot_data_set(x, y)
plot_decision_region(x[:, 0].min(), x[:, 1].min(), x[:, 0].max(), x[:, 1].max(), clf)
pylab.show()

## Концентрические окружности

In [ ]:
def generate_concentric(size=100, r1=1.0, r2=2.0, sigma=0.3):
    print "Generating 'Concentric circles' data set"
    x = numpy.zeros((size, 2))
    x[:size/2, 0] = sigma * numpy.random.randn(size/2) + r1
    x[size/2:, 0] = sigma * numpy.random.randn(size/2) + r2
    x[:, 1] = (numpy.random.random(size) - 0.5) * 2 * numpy.pi
    y = numpy.hstack([numpy.zeros(size/2, dtype=int), numpy.ones(size/2, dtype=int)])

    z = numpy.zeros((size, 2))
    z[:, 0] = x[:, 0] * numpy.cos(x[:, 1])
    z[:, 1] = x[:, 0] * numpy.sin(x[:, 1])

    return z, y

x, y = generate_concentric()
clf = select_model(x, y)
plot_data_set(x, y)
plot_decision_region(x[:, 0].min(), x[:, 1].min(), x[:, 0].max(), x[:, 1].max(), clf)
pylab.show()

## Смесь гауссовских распределений

In [ ]:
def generate_gauss(size=200):
    print "Generating 'Gaussian mixture' data set"

    x, y = sklearn.datasets.make_classification(n_samples=size,
                                                n_features=2,
                                                n_informative=2,
                                                n_redundant=0,
                                                n_classes=2,
                                                n_clusters_per_class=2)
    return x, y

x, y = generate_gauss()
clf = select_model(x, y)
plot_data_set(x, y)
plot_decision_region(x[:, 0].min(), x[:, 1].min(), x[:, 0].max(), x[:, 1].max(), clf)
pylab.show()

## Синусоидальная граница

In [ ]:
def generate_sin(size=200):
    print "Generating 'Sinus-separated' data set"

    x = numpy.random.random((size, 2))
    x[:, 0] = x[:, 0] * 4 * numpy.pi
    x[:, 1] = (x[:, 1] - 0.5) * 2
    y = numpy.zeros(size, dtype=int)
    y[x[:, 1] > numpy.sin(x[:, 0])] = 1

    return x, y

x, y = generate_sin()
clf = select_model(x, y)
plot_data_set(x, y)
plot_decision_region(x[:, 0].min(), x[:, 1].min(), x[:, 0].max(), x[:, 1].max(), clf)
pylab.show()

## Digits

Набор данных состоящий из 1797 изображений, представленных матрицей 8x8. Каждое избражение -- написанная цифра. 

Подробнее здесь -- http://archive.ics.uci.edu/ml/datasets/Pen-Based+Recognition+of+Handwritten+Digits

In [ ]:
digits = datasets.load_digits()
images_and_labels = list(zip(digits.images, digits.target))
for index, (image, label) in enumerate(images_and_labels[:4]):
    plt.subplot(2, 4, index + 1)
    plt.axis('off')
    plt.imshow(image, cmap=plt.cm.gray_r, interpolation='nearest')
    plt.title('Training: %i' % label)

Чтобы классифицировать эти объекты переведем матрицу 8x8 в строку длинной 64 для каждого изображения.

In [ ]:
def matrix_to_string(data):
    #реализуйте функцию, которая для каждого изображения отображает матрицу 8x8 в строку длинной 64
    return converted_data

In [ ]:
X = matrix_to_string(digits.images)
y = digits.target

Разделите данные на обучающую и тестовую выборки

In [ ]:
def split_data(X, y, test_size):
    #реализуйте функцию, которая разделит данные на обучающие и тестовые
    return X_train, X_test, y_train, y_test


In [ ]:
X_train, X_test, y_train, y_test = split_data(X, y, 0.33)

Обучаем классификатор

In [ ]:
clf = select_model(X_train, y_train)

Предскажите метку для тестовых данных

In [ ]:
def predict(clf, X_test):
    #реализуйте функцию, которая вычисляет предсказания для тестовой выборки
    return prediction
predicted = predict(clf, X_test)

In [ ]:
print("Classification report for classifier %s:\n%s\n"
      % (clf, metrics.classification_report(y_test, predicted)))
print("Confusion matrix:\n%s" % metrics.confusion_matrix(y_test, predicted))

images_and_predictions = list(zip([X_test[i].reshape((8,8)) for i in range(len(X_test))], predicted))
for index, (image, prediction) in enumerate(images_and_predictions[:4]):
    plt.subplot(2, 4, index + 5)
    plt.axis('off')
    plt.imshow(image, cmap=plt.cm.gray_r, interpolation='nearest')
    plt.title('Prediction: %i' % prediction)

plt.show()